# Group 2315: Analyzing Plasma Evolution with Transformers

#### Members:
- Amirhossein Rostami
- Davide Checchia
- Jelin Raphael Akkara - 2072064
- Kiamehr Javid

## Defining Libraries

In [1]:
#import tensorflow as tf
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.utils.class_weight as class_weight
#from tensorflow import keras
#from tensorflow.keras import backend as K
#from keras import layers
#from keras.callbacks import LearningRateScheduler
from matplotlib.lines import Line2D
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import KFold, train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
#from keras.layers import *
#from keras.models import *
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

#import os
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
tf.config.list_physical_devices('GPU')

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda/'

## Loading Data

In [242]:
#Returns x with shape(Total Valid Windows, Window length, Features), y with shape(Total Valid Windows,)
def load_timeseries(path, cols_select, train_window=150, pred_time_gap=10, pred_window=0, acceptance_threshold=0.2, 
                    plot_param=False, plot_param_pos=2, 
                    pca=True, pca_num=15, plot_pca_var=False):  
                    
    data = np.genfromtxt(path, delimiter=',', skip_header=1)
    colnames = pd.read_csv(path, index_col=0, nrows=0).columns.tolist()
    x, y = [], []

    for i in range(len(data)-train_window-pred_time_gap):
      label = float(1.0 in data[i+train_window+pred_time_gap-pred_window : i+train_window+pred_time_gap+1, 3])        #Checks if 1 occurs in pred_window cells after pred_time_gap steps

      if label == 0 and np.random.uniform(0, 1) > acceptance_threshold: continue  #Accepts a certain percentage of zero labels randomly

      if 1.0 not in data[i:i+train_window+pred_time_gap-pred_window-1, 3]:                                          #If window does not have crash as 1 in it, append
          if cols_select == ':': 
            col_unwanted = [0, 1]                                                       #Unwanted feature column indices
            col_indices = [i for i in range(0, data.shape[1]) if i not in col_unwanted] #Selects all column indices except unwanted ones
            x.append(data[i:i+train_window, col_indices])                               #Selects all features
          else: x.append(data[i:i+train_window, cols_select])                           #Only takes specific (cols_select) features          
          y.append(label)
          if len(y) == 1: prim_arg = i

    if len(np.shape(x)) == 2: x = np.expand_dims(x, axis=2)       #If only one feature, expand dimension
    x, y = np.array(x), np.array(y)


    # Applying PCA 
    if pca: 

      #Shape of x before PCA
      print("Shape of the x before PCA",x.shape)    

      if pca_num > len(cols_select) and cols_select != ':': 
        print('Invalid Entry: The pca_num is greater than number of selected features.')
        return()

      reshaped_x = np.reshape(x, (x.shape[0]*x.shape[1], x.shape[2]))

      scaler = StandardScaler()                       #Standardizes the features
      data_scaled = scaler.fit_transform(reshaped_x)
      pca = PCA(n_components=pca_num)                 # Specify the number of components you want to keep

      x_pca = pca.fit_transform(data_scaled)
      var_ratios = pca.explained_variance_ratio_      #Gets variance contribution of each pca component
      #print('Variance Contribution of each pca component: ', var_ratios)
      print('Total Variance covered: ', sum(var_ratios))

      if plot_pca_var:  #Plots progression of variance covered as number of chosen components change
        plt.plot(np.cumsum(var_ratios))
        plt.xlabel('Number of Components Chosen')
        plt.ylabel('Variance Covered')
        plt.grid(True)
        plt.title('Variance covered wrt Components Chosen')

      x = np.reshape(x_pca, (x.shape[0],x.shape[1], pca_num))
      print("PCA Successful, Shape of the pca data is : ",x.shape)

    #Plotting
    if plot_param:
      time_vals = data[:, 1]
      param_vals = data[:, plot_param_pos]
      crash_times = data[data[:,3] == 1][:, 1]    #Times at which crashes occur

      fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(25, 10), gridspec_kw={'height_ratios': [3, 1]})
      fig.tight_layout(pad=7)

      #First Plot - Plotting Parameter evolution
      ax[0].plot(time_vals, param_vals)
      for t in crash_times: ax[0].axvline(t, color='red', linestyle='--', lw=0.5, alpha=0.5)  #Plotting markers for crashes
      ax[0].set_title('Time vs '+colnames[plot_param_pos-1], fontweight='bold')

      #Second plot - Plotting training window, prediction window
      ax[1].plot(time_vals, param_vals)
      ax[1].set_xlim((min(time_vals)-500, time_vals[prim_arg+train_window+pred_time_gap]+500))  #Restrictin x range to windows
      ax[1].set_title('Training and Prediction Windows', fontweight='bold')

      #Plotting x window, prediction window in both plots
      for pos in [0, 1]:            
            window_t_low = time_vals[prim_arg]
            window_t_up = time_vals[prim_arg + train_window]

            ax[pos].axvline(window_t_low, color='black', lw=2, ls='--',  alpha=0.7, label='Train Window')           #Lower limit of window
            ax[pos].axvline(window_t_up, color='black', lw=2, ls='--', alpha=0.7)                                   #Upper limit of window
            ax[pos].axvspan(window_t_low, window_t_up, alpha=0.3, color='gray')                                     #Shades window region
            
            for i in range(pred_window+1):  #Upper limit of pred_time_gap
              ax[pos].axvline(time_vals[prim_arg+train_window+pred_time_gap-i], color='darkgreen', lw=2, ls='--', alpha=1, label='Prediction')  

            ax[pos].set_xlabel('Time (Alfven)')
            ax[pos].set_ylabel(colnames[plot_param_pos-1])

      #Plotting Legend
      handles, labels = plt.gca().get_legend_handles_labels()   #Gets handles and labels

      #Manually appending crash line to handles and labels
      crash_line = Line2D([0], [0], color='red', linestyle='--')
      handles.extend([crash_line])    
      labels.extend(['Crash'])

      by_label = dict(zip(labels, handles))   #Only takes in unique values
      ax[0].legend(by_label.values(), by_label.keys(), loc='upper right')

    return x, y

In [3]:
Length = 31              #Window Length
delta_t = 30             #Prediction length
WR = 1                   #Prediction Window
acceptance_perc = 0.2    #Percentage of zeros selected (Down-sampling)
training_perc = 0.7      #Percentage used for training

path = "/content/drive/MyDrive/LCP_Project/merged_file.csv"

x, y = load_timeseries(path, ':', 
                       train_window=Length, 
                       pred_time_gap = delta_t,                    
                       pred_window = WR,
                       acceptance_threshold= acceptance_perc, 
                       plot_param=True, 
                       plot_param_pos=2, 
                       pca=False, 
                       pca_num=29,
                       plot_pca_var=False)
print(x.shape, y.shape)

------------------

## Splitting to training and testing sets

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, train_size=training_perc)

num_classes = len(np.unique(y_train))
print('Unique Classes: ', num_classes)

rep = 0

for i in range(rep):
  x_train = np.concatenate((x_train,x_train[y_train==1]))
  y_train = np.concatenate((y_train,y_train[y_train==1]))

  idx = np.random.permutation(len(x_train))
  x_train = x_train[idx]
  y_train = y_train[idx]

w = sum(y_train)/len(y_train)           #Gets proportion of ones in training set

Unique Classes:  2


In [7]:
zeros_train, ones_train = np.bincount(y_train.astype(int))   #Gets counts of 0, 1 in training set
zeros_test, ones_test = np.bincount(y_test.astype(int))   #Gets counts of 0, 1 in testing set

ones_train_perc = 100*(ones_train/len(y_train))   #Percentage division in training set
zeros_train_perc = 100 - ones_train_perc          

ones_test_perc = 100*(ones_test/len(y_test))    #Percentage division in testing set
zeros_test_perc = 100 - ones_test_perc

print('Percentage of 1 in training set: ', ones_train_perc)
print('Percentage of 0 in training set: ', zeros_train_perc)

print('\nPercentage of 1 in testing set: ', ones_test_perc)
print('Percentage of 0 in testing set: ', zeros_test_perc)

Percentage of 1 in training set:  1.604434072345391
Percentage of 0 in training set:  98.39556592765462

Percentage of 1 in testing set:  1.6164709885996258
Percentage of 0 in testing set:  98.38352901140037


--------------

## Define Metrics

In [ ]:
metrics_list = [
      #keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      #keras.metrics.MeanSquaredError(name='Brier score'),
      # keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      'accuracy',
    #   keras.metrics.BinaryAccuracy(name='bin_accuracy'),
       keras.metrics.Precision(name='precision'),
       keras.metrics.Recall(name='recall'),
    #   keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

-------------------

## Define Models and Compile

#### Transformer Model

In [ ]:
# TRANSFORMER
shape = x_train.shape[1:]



def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0, kernel_size=1 , act = 'gelu'):
    # Normalization and Attention
    x = inputs
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=kernel_size, activation=act, padding="same")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=kernel_size, padding="same")(x)
    return x + res


def build_model(
    input_shape=shape,
    head_size=256,
    num_heads=1,
    ff_dim=16,
    kernel_size=1,
    num_transformer_blocks=3,
    mlp_units=[],
    act = 'gelu',
    dropout=0.5,
    mlp_dropout=0,
):
    K.clear_session()
    inputs = keras.Input(shape=(input_shape))
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout, kernel_size , act)

    x = layers.Flatten(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation=act)(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs) 

    model.compile(loss = 'binary_crossentropy' , metrics = metrics_list , optimizer = keras.optimizers.Adam(learning_rate=1e-3))
    return model


model = build_model(shape,
    head_size = 256,
    num_heads = 1,
    ff_dim = 16,
    kernel_size=1,
    num_transformer_blocks = 3,
    mlp_units=[],
    mlp_dropout=0.5,
    dropout=0.5)

model.summary()

#### CNN Model

In [8]:
def make_cnn_model(input_shape = x_train.shape[1:] , num_layers = 3 , kernel = 5 , filters = 16 , do_batch_norm = True , act = 'relu',dropout= 0 ):    
    input_layer = keras.layers.Input(input_shape)

    l = input_layer

    for _ in range(num_layers):

      l = keras.layers.Conv1D(filters=filters, kernel_size=kernel, padding="same")(l)
      l = keras.layers.Dropout(dropout)(l)
      if do_batch_norm : l = keras.layers.BatchNormalization()(l)
      l = keras.layers.Activation(act)(l)   #activation for layers 
      
    gap = keras.layers.Flatten()(l)   #Flattens
    #gap = keras.layers.GlobalAveragePooling1D()(conv1)

    output_layer = keras.layers.Dense(1, activation='sigmoid')(gap) #activation fro output layer 

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=8*1e-3),
    loss=["binary_crossentropy"],
    
    metrics=[keras.metrics.TruePositives(name='tp'), 
             keras.metrics.BinaryAccuracy(name='bin_accuracy'), 
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')]
    )
    return model

model = make_cnn_model(x_train.shape[1:])
model.summary()

#### DNN Model

In [ ]:
def make_dnn(input_shape=x_train.shape[1:] , dims = [2038,1024,512,256,128,64,32,16,8] , act = 'elu' , dropout = 0):
  
  
  inputs = input_layer = keras.layers.Input(input_shape)
  l = inputs

  l = keras.layers.Flatten()(l)  
  for dim in dims:
    l = keras.layers.Dense(dim , activation = act )(l)
    l = keras.layers.Dropout(dropout)(l)
  output = keras.layers.Dense(1,activation = 'sigmoid')(l)

  model = keras.models.Model(inputs=inputs, outputs=output)

  model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=["binary_crossentropy"],
    metrics=[keras.metrics.TruePositives(name='tp'), 
             keras.metrics.BinaryAccuracy(name='bin_accuracy'), 
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')]
             )

  return model


model = make_dnn(x_train.shape[1:])
model.summary()

-----------------

## Validating Model (Stratified K-Fold Cross Validation)

In [ ]:
param_grid = {'filters': [8, 16, 32, 64], 
              'kernel': [1, 3, 4, 6], 
              'num_layers': [1, 3, 6]}

def lr_schedule(epoch, lr):
    if epoch % 3 == 0 and epoch > 0:
        lr = lr / 2
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

#Using class_weights assigned by keras
#class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes=np.unique(y_train), y = y_train) #Assigns weights to each class wrt counts
#class_weights = {i : class_weights[i] for i in range(len(class_weights))}   #Class weight dictionary

model_search = KerasClassifier(build_fn=make_cnn_model, verbose=2 , epochs = 30 , callbacks = [lr_scheduler] , class_weight = {0:w*10,1:(1-w)*10})
grid_search = GridSearchCV(estimator=model_search, param_grid=param_grid, cv=3 ,scoring=["accuracy", "recall", "roc_auc"], refit="accuracy" , verbose = 3)


res = grid_search.fit(x_train, y_train , verbose = 2)

In [ ]:
#Save Grid Search Results

results = pd.DataFrame(res.cv_results_)
# results.sort_values(by='accuracy', inplace=True)
results.to_excel("DNN_GS_res.xlsx")
results

## Training Model

In [ ]:

def lr_schedule(epoch, lr):
    if epoch % 6 == 0 and epoch > 0:
        lr = lr / 2
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

#class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes=np.unique(y_train), y = y_train) #Assigns weights to each class wrt counts
#class_weights = {i : class_weights[i] for i in range(len(class_weights))}   #Class weight dictionary

history = model.fit(x_train , y_train , 
                        validation_split = 0.1 , 
                        epochs = 60 , 
                        callbacks = lr_scheduler , 
                        batch_size = 200 , 
                        class_weight = {0:w,1:1-w})

## Testing Model

In [15]:
#Predictions from model with training and testing sets
batch_size=200
pred_train = model.predict(x_train, batch_size=batch_size)
pred_test = model.predict(x_test, batch_size=batch_size)
print(np.shape(pred_train), np.shape(pred_test))

40/40 [==============================] - 0s 3ms/step
(13712, 1) (5877, 1)


In [ ]:
results = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

for metric, value in zip(model.metrics_names, results): #Prints metric scores
  print(f'{metric}: {value}')

loss: 0.04561721161007881
tp: 66.0
accuracy: 0.0
bin_accuracy: 0.9902480840682983
precision: 0.7021276354789734
recall: 0.6947368383407593
auc: 0.9163697957992554


-------------------

## Post-Processing Results

In [2]:
def get_matrix_df(df, param_colnames, score_colname):       #Converts df to matrix, unique param values for heatmap

    fil_cols = param_colnames + [score_colname]             #Gets list of column names to be filtered from DataFrame
    df_fil = df[fil_cols].sort_values(param_colnames)       #Gets filtered Dataframe, sorts according to ordered param list

    distinct_param_vals = [np.unique(df_fil[col]) for col in param_colnames]    #Gets distinct param values for each param

    mat_dim = tuple([len(pvals) for pvals in distinct_param_vals])              #Matrix dimension wrt number of unique param values
    matrix = np.array(df_fil[score_colname]).reshape(mat_dim)                   #Gets ordered matrix

    return matrix, distinct_param_vals

In [3]:
#Converts text file to data frame
def text_to_df(path, param_indices, add_fbeta=True, beta=3):

    score_dict = {'L': [], 'dt': [], 'accept_perc': [], 'WR': [], 'pca': [],
                'loss': [], 'tp': [], 'accuracy': [], 'recall': [], 'precision': []}

    with open(path, 'r') as file:
        vals = file.readlines()[6:]         #Gets each line in the file into a list

    for lcount, line in enumerate(vals):    #Iterates through each line
        if line == '\n': continue           #Ignores new line statement
        line = re.sub(',', '', line)        #Removes ','
        line_split = line.split()           #Splits line string into list of elements

        if line_split[0] == 'MODEL':         

            for i in range(len(param_indices)):     #Gets param values (L, dt, accept_perc, WR, pca)
                key = list(score_dict.keys())[i]    #Corresponding key in score_dict wrt iteration

                if i == 2: score_dict[key].append( float( line_split[param_indices[i]][:-1] ) / 100)    #Gets accept_perc as float
                elif i == 4: score_dict[key].append( ( line_split[param_indices[i]] ))                  #Gets pca as string
                else : score_dict[key].append( int( line_split[param_indices[i]] ))                     #Gets rest of metrics as integer

            for i in range(0, 5):                   #Gets metric values (loss, tp, accuracy, recall, precision)
                key = list(score_dict.keys())[i+5]  #Corresponding key in score_dict wrt iteration
                score_dict[key].append(float( vals[lcount+i+1].split()[1] ))    #Gets value as float
    
    df = pd.DataFrame(score_dict)   #Converts to Dataframe
    if add_fbeta:                   #Adds f_beta score column
        df['fbeta_score'] = (1+beta**2) / ((beta**2)*(1/df['recall']) + (1/df['precision']))

    return df


In [239]:
#Obtaining dataframe from txt file
files = ['L_dt_PCAFalse_WR0.txt', 'L_dt_PCATrue_WR0.txt', 'L_dt_PCAFalse_WR1.txt', 'L_dt_PCATrue_WR1.txt']
path = 'Results/Transformers/L_dt_PCA_WR/'
param_indices = [4, 7, 10, 13, 16]      #Positions of metrics in string
beta_val=5                              #Initializes beta value for F_score 

df_w0_pca0 = text_to_df(path+files[0], param_indices, add_fbeta=True, beta=beta_val).groupby(by=['L', 'dt']).mean().reset_index()
df_w0_pca1 = text_to_df(path+files[1], param_indices, add_fbeta=True, beta=beta_val).groupby(by=['L', 'dt']).mean().reset_index()
df_w1_pca0 = text_to_df(path+files[2], param_indices, add_fbeta=True, beta=beta_val).groupby(by=['L', 'dt']).mean().reset_index()
df_w1_pca1 = text_to_df(path+files[3], param_indices, add_fbeta=True, beta=beta_val).groupby(by=['L', 'dt']).mean().reset_index()

In [240]:
#For Heatmap: Getting matrix, unique param values for each case
matrix_00, pvals_00 = get_matrix_df(df_w0_pca0, ['L', 'dt'], 'fbeta_score')
matrix_01, pvals_01 = get_matrix_df(df_w0_pca1, ['L', 'dt'], 'fbeta_score')
matrix_10, pvals_10 = get_matrix_df(df_w1_pca0, ['L', 'dt'], 'fbeta_score')
matrix_11, pvals_11 = get_matrix_df(df_w1_pca1, ['L', 'dt'], 'fbeta_score')

In [126]:
#For Line Plot:  Adding column 'total window' and grouping wrt it
df_w0_pca0['total_window'] = df_w0_pca0['L'] + df_w0_pca0['dt']
df_w0_pca1['total_window'] = df_w0_pca1['L'] + df_w0_pca1['dt']
df_w1_pca0['total_window'] = df_w1_pca0['L'] + df_w1_pca0['dt']
df_w1_pca1['total_window'] = df_w1_pca1['L'] + df_w1_pca1['dt']

df_w0_pca0 = df_w0_pca0.groupby(by=['total_window']).mean().reset_index()
df_w0_pca1 = df_w0_pca1.groupby(by=['total_window']).mean().reset_index()
df_w1_pca0 = df_w1_pca0.groupby(by=['total_window']).mean().reset_index()
df_w1_pca1 = df_w1_pca1.groupby(by=['total_window']).mean().reset_index()

----------------------

## Plotting

#### Plotting: Metric from model history

In [299]:
def plot_metric(metric, y_vals, y_type='Train', div_y=False, add_plot=False):  #Plots metric progression against all y values

  if div_y:   #Divides metric values by total y_vals
    plt.plot(history.history[metric]/sum(y_vals), color='#1f77b4')          #Plots training curve
    plt.plot(history.history['val_'+metric]/sum(y_vals), color='orange')    #Plots validation curve
  else: 
    plt.plot(history.history[metric], color='#1f77b4')
    plt.plot(history.history['val_'+metric], color='orange')

  plt.grid(True)
  plt.title(f'Model {metric}')
  plt.ylabel(metric, fontsize="large")
  plt.xlabel("epoch", fontsize="large")
  plt.legend([y_type, "Validation"], loc="lower right")


In [300]:
def plot_roc(name, labels, predictions, **kwargs):    #Plots ROC Curve
  fp, tp, _ = roc_curve(labels, predictions)          #Gets false positives, true positives

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, color='blue', **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  #plt.xlim([-0.5,20])
  #plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
#Plotting metric
plot_metric('recall', y_train, 'Train', div_y=False)

In [ ]:
#Plotting ROC Curve
plot_roc('Train Curve', y_train, pred_train)
plot_roc('Test Curve', y_test, pred_test, linestyle='--')
plt.legend(loc='lower right')

#### Plotting: Confusion Matrix for model

In [323]:
def plot_cm(ax, labels, predictions, data_type, threshold=0.5, color_simple=False, label_order = [1, 0]):    #Plots confusion matrix

    cm = confusion_matrix(labels, predictions > threshold, labels=label_order)    #Gets confusion matrix counts as array

    row_sums = np.sum(cm, axis = 1)                           #Gets sum along matrix rows
    row_sums_mat = np.tile(row_sums, (cm.shape[0], 1)).T      #Converts row sum values into shape of confusion matrix
    cm_percs = cm / row_sums_mat                              #Gets confusion matrix with rows divided by row sums

    cm_percs_str = ["{0:.2%}".format(perc) for perc in cm_percs.flatten()]                          #Flattens cm, expresses percents in 2 decimal places
    annot_labels = [f'  {perc} \n ( {count} )' for perc, count in zip(cm_percs_str, cm.flatten())]  #Prepares annotation labels
    annot_labels = np.array(annot_labels).reshape((2, 2))                                           #Gets annot labels in shape of confusion matrix
  
    if color_simple: cmap = sns.color_palette('Blues', as_cmap=True)  #One dimensional Color Map
    else:                                                             #Diverging Color Map
      cmap = sns.color_palette("vlag", as_cmap=True)                            
      cmap = list(reversed(cmap.colors))

    #Plots confusion matrix using heatmap, label order is the ordering of axis labels ((1, 0) or (0, 1))
    hm = sns.heatmap(cm_percs*100, annot=annot_labels, fmt="", cmap=cmap, ax=ax, yticklabels=label_order, xticklabels=label_order)                    

    hm.set_title(f'{data_type} : Confusion matrix with threshold = {threshold}', fontsize=11, pad=10)
    hm.set_ylabel('Actual label')
    hm.set_xlabel('Predicted label')

In [ ]:
#Plotting only Training Set
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 7))
plot_cm(ax, y_train, pred_train, data_type='Train Data', threshold=0.5, color_simple=False, label_order = [1, 0])

In [ ]:
#Plotting both training and testing set
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))
fig.tight_layout(pad=7)

plot_cm(ax[0], y_train, pred_train, data_type='Train Data', threshold=0.5, color_simple=False, label_order = [1, 0])
plot_cm(ax[1], y_test, pred_test, data_type='Test Data', threshold=0.5, color_simple=True, label_order = [1, 0])

#### Plotting: Threshold Analysis -
- Plotting Histogram of prediction values
- Plotting progression of precision, recall, F_score
- Plotting progression of True Positives, True Negatives, False Positives, False Negatives

In [302]:
def get_opt_threshold(pred_vals, beta_val=5, plot_precall=False, plot_each=False, plot_hist=False, n_bins=10, yrange=(0, 500)): #Gets optimal threshold
  threshold_vals = np.linspace(0, 1, 100)[1:-1]         #Array of threshold values to plot over
  precision_vals, recall_vals = [], []              
  tp_vals, tn_vals, fp_vals, fn_vals = [], [], [], []

  for thr_val in threshold_vals:    #Gets metric values for each threshold
    cm = confusion_matrix(y_train, pred_vals > thr_val, labels=[1, 0])
    tp, tn, fp, fn = cm[0, 0], cm[1, 1], cm[1, 0], cm[0, 1]

    if plot_each: 
      tp_vals.append(tp)
      tn_vals.append(tn)
      fp_vals.append(fp)
      fn_vals.append(fn)

    precision_vals.append( tp / (tp + fp) )
    recall_vals.append( tp / (tp + fn) )

  #Inverses precision, recall values
  precision_inv = 1/np.array(precision_vals)
  recall_inv = 1/np.array(recall_vals)
  
  fbeta_vals = (1 + beta_val**2) / ((beta_val**2 * recall_inv) + precision_inv)   #Gets F_beta score wrt given beta value
  f_arg_max = np.argwhere(fbeta_vals == max(fbeta_vals))                          #Gets argument for maximum F_beta score
  opt_threshold = threshold_vals[f_arg_max][0, 0]                                 #Gets maximum threshold

  print('Threshold with max f value: ', opt_threshold)

  if plot_hist:   #Plots histogram of prediction values
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8, 6), gridspec_kw={'height_ratios': [3, 1]})
    fig.tight_layout(pad=1)

    pred_flat = pred_vals.flatten()
    ax[0].hist(pred_flat, bins=n_bins, alpha=0.9, range=(0, 1))   #Plots histogram of predicted values
    ax[0].set_title('Histogram: Training Predictions')
    ax[0].grid(True, alpha=0.5)
    ax[0].set_ylabel('Counts')

    ax[1].hist(pred_flat, bins=n_bins, alpha=0.9, range=(0, 1))   #Plots close-up version for low count values
    ax[1].grid(True, alpha=0.5)
    ax[1].set_ylim(yrange)
    ax[1].set_xlabel('Prediction')
    ax[1].set_ylabel('Counts')


  if plot_precall:    #Plots progression of precision, recall, F_beta score wrt threshold
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    fig.tight_layout(pad=5)

    ax[0].plot(threshold_vals, recall_vals, label='Recall')         #Plots recall curve
    ax[0].plot(threshold_vals, precision_vals, label='Precision')   #Plots precision curve
    ax[0].set_ylabel('Value')
    ax[0].set_title('Variation across Thresholds')
    ax[0].legend()

    ax[1].plot(threshold_vals, fbeta_vals)
    ax[1].set_ylabel('F Value')
    ax[1].set_title(f'F-{beta_val} Metric across Thresholds')
    ax[1].text(opt_threshold-0.05, 0.5, 'Optimal Threshold: {:.2f}'.format(opt_threshold), rotation=90)

    for i in [0, 1]:
        ax[i].grid(True, alpha=0.7)
        ax[i].set_xlabel('Threshold')
        ax[i].axvline(opt_threshold, color='red', linestyle='--', linewidth=0.8)

  if plot_each:     #Plots progression of (tp, fn, tn, fp) wrt threshold
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
    fig.tight_layout(pad=5)

    ax[0, 0].plot(threshold_vals, tp_vals)
    ax[0, 0].set_title('True Positives')

    ax[0, 1].plot(threshold_vals, fn_vals, color='firebrick')
    ax[0, 1].set_title('False Negatives')

    ax[1, 0].plot(threshold_vals, fp_vals, color='firebrick')
    ax[1, 0].set_title('False Positives')

    ax[1, 1].plot(threshold_vals, tn_vals)
    ax[1, 1].set_title('True Negatives')

    for i in [0, 1]:
      for j in [0, 1]:
        ax[i, j].grid(True, alpha=0.7)
        ax[i, j].set_xlabel('Threshold')
        ax[i, j].set_ylabel('Counts')

    return (opt_threshold)

In [ ]:
opt_threshold = get_opt_threshold(pred_train, beta_val=15, plot_precall=False, plot_each=True, plot_hist=False)

#### Plotting: Heatmap

In [219]:
def plot_heatmap(ax, matrix_vals, param0_vals, param1_vals,     #Plots 2D heatmap given ordered matrix, unique param values
                    title, xlabel, ylabel, cbar_label,
                    normalize_mat=False, 
                    flip_vert=False, 
                    color=0, invert_cmap=False,
                    annotate=False,
                    highlight_max_cells=3):     

    if normalize_mat:       #Normalizes matrix with max, min values
        max_val = np.max(matrix_vals)
        min_val = np.min(matrix_vals)

        norm_matrix = (matrix_vals - min_val) / (max_val - min_val)
        matrix_vals = norm_matrix

        title = title + " \n ( Max: {0:.2f}, Min: {1:.2f} )".format(max_val, min_val)

    if flip_vert:           #Flips matrix vertically
        matrix_vals = np.flipud(matrix_vals)
        param0_vals = param0_vals[::-1]

    #Choosing color
    if color == 0:   cmap = sns.color_palette("vlag", as_cmap=True)    
    elif color == 1: cmap = sns.diverging_palette(220, 20, as_cmap=True)      
    elif color == 2: cmap = sns.color_palette("flare", as_cmap=True)
    else: cmap = sns.diverging_palette(250, 30, l=65, center="dark", as_cmap=True)
    
    if invert_cmap: cmap = cmap.reversed()  #Inverts color map

    #Plots heatmap
    hm = sns.heatmap(matrix_vals, annot=annotate, cmap=cmap, ax=ax,
                        xticklabels=param1_vals, yticklabels=param0_vals,
                        vmin=0, vmax=1)

    hm.set_title(title, fontweight='bold')
    hm.set_xlabel(xlabel)
    hm.set_ylabel(ylabel)

    cbar = ax.collections[0].colorbar
    cbar.set_label(cbar_label, labelpad=10)

    if highlight_max_cells !=0:     #Highlights Cells with n maximum values
        rounded_matrix = np.round(matrix_vals.flatten(), 5)                     #Rounds matrix for comparison, avoiding floating point errors
        max_vals = np.sort(np.unique(rounded_matrix))[-highlight_max_cells:]    #Gets n maximum values
        max_args = [np.where(rounded_matrix == val)[0] for val in max_vals]     #Gets corresponding arguments for the maximum values

        for args in max_args:
            for ind in args:
                row, col = np.unravel_index(ind, matrix_vals.shape)

                hm.add_patch(plt.Rectangle((col, row), 1, 1, fill=False,    #Highlights border of chosen cell
                                edgecolor='black', lw=2, linestyle='--'))

In [ ]:
#Plotting L-dt Heatmap (F Score as measure)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.tight_layout(pad=8)
fig.suptitle(f'Plotting L vs dt \n (Normalized F_{beta_val} Score as measure)', fontweight='bold', fontsize=15)
plt.subplots_adjust(top=0.89)

axes_list = [[0, 0], [0, 1], [1, 0], [1, 1]]
matrix_list = [matrix_00, matrix_01, matrix_10, matrix_11]
pvals_list = [pvals_00, pvals_01, pvals_10, pvals_11]

for i in range(len(axes_list)):
    plot_heatmap(ax[axes_list[i][0], axes_list[i][1]], 
                    matrix_list[i], pvals_list[i][0], pvals_list[i][1],
                    f'Heatmap (WR:{axes_list[i][0]}, PCA:{axes_list[i][1]}, Beta: {beta_val})', 
                    'dt', 'L', f'F{beta_val}_score', 
                    normalize_mat=True, flip_vert=True, 
                    color=1, invert_cmap=True,
                    highlight_max_cells=3,
                    annotate=True)

In [ ]:
#Plotting L-dt Heatmap (Recall as measure)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.tight_layout(pad=6)
fig.suptitle('Plotting L vs dt \n (Recall as measure)', fontweight='bold', fontsize=15)
plt.subplots_adjust(top=0.89)

axes_list = [[0, 0], [0, 1], [1, 0], [1, 1]]
matrix_list = [matrix_00, matrix_01, matrix_10, matrix_11]
pvals_list = [pvals_00, pvals_01, pvals_10, pvals_11]

for i in range(len(axes_list)):
    plot_heatmap(ax[axes_list[i][0], axes_list[i][1]], 
                    matrix_list[i], pvals_list[i][0], pvals_list[i][1],
                    f'Heatmap (WR:{axes_list[i][0]}, PCA:{axes_list[i][1]})', 
                    'dt', 'L', 'recall', 
                    normalize_mat=False, flip_vert=True, 
                    color=1, invert_cmap=True,
                    highlight_max_cells=3,
                    annotate=True)

#### Plotting: F_Score reference plots with recall, precision for given beta value

In [295]:
def plot_fbeta_ref(ax, beta=5, accuracy=0.1, annotate=False, color=0):  #Plots reference heatmap with recall vs precision for a given beta value

    def f_beta(beta, precision, recall):        #Gets F_beta score
        if recall == 0 or precision == 0: return 0
        fbeta = (1+beta**2) / (((beta**2)/recall) + 1/precision)
        return fbeta

    precision_vals = np.round(np.arange(0, 1+accuracy, accuracy), 5)    #Array of precision values for heatmap
    recall_vals = np.round(np.arange(0, 1+accuracy, accuracy), 5)       #Array of recall values for heatmap
    fbeta_vals = [f_beta(beta, prec, recall) for prec in precision_vals for recall in recall_vals]  #Gets f_beta score for all combinations of precision, recall

    matrix_fbeta = np.array(fbeta_vals).reshape((len(precision_vals), len(recall_vals)))    #Expresses fbeta_vals as ordered matrix

    plot_heatmap(ax, matrix_fbeta, precision_vals, recall_vals, 
                title=f'Matrix Heatmap: (Beta: {beta})', xlabel='recall', ylabel='precision', cbar_label='F5_score',
                normalize_mat=False,
                flip_vert=True,
                color=color,
                invert_cmap=True,
                annotate=annotate,
                highlight_max_cells=0)

In [ ]:
#Plotting F_Beta Reference Heatmaps
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.tight_layout(pad=6)

axes_list = [[0, 0], [0, 1], [1, 0], [1, 1]]
beta_vals = [0, 1, 2, 3]

for i in range(len(axes_list)):
    plot_fbeta_ref(ax[axes_list[i][0], axes_list[i][1]], beta_vals[i], 
                   annotate=True, color=1)

#### Plotting: Line Plot

In [97]:
def plot_df_line(ax, df, param0, param1, title, regress=False,  yrange=(0, 1)):     #Line plot given dataframe and parameter column names
    #Plots line plot
    ax.plot(df[param0], df[param1])

    if regress: #Plots regression fit
        sns.regplot(data=df, x=param0, y=param1, ax=ax, 
                        order=1,    #Order of regression polynomial
                        ci=95,      #Confidence Interval
                        marker='o',
                        line_kws={'linewidth': 2, 'linestyle': '--', 'alpha': 0.7}
                        )

    ax.set_xlabel(param0)   
    ax.set_ylabel(param1)
    ax.set_ylim(yrange)
    ax.set_title(title)
    ax.grid(True)

In [ ]:
#Plotting Line Plots: F_score vs dt
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.tight_layout(pad=6)
fig.suptitle(f'Plotting F_{beta_val} score vs dt', fontweight='bold', fontsize=15)

axes_list = [[0, 0], [0, 1], [1, 0], [1, 1]]
df_list = [df_w0_pca0, df_w0_pca1, df_w1_pca0, df_w1_pca1]

for i in range(len(axes_list)):
    plot_df_line(ax[axes_list[i][0], axes_list[i][1]],      #Mentioning Axis of figure
                    df_list[i], 'dt', 'fbeta_score',             #Dataframe, params to be plotted
                    regress=True,
                    title=f'(W: {axes_list[i][0]}, pca: {axes_list[i][1]}, L: 45)', 
                    #yrange=(0.2, 0.8)
                )

In [ ]:
#Plotting Line Plots: Total Window vs Recall
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.tight_layout(pad=6)
fig.suptitle(f'Plotting Total Window vs Recall', fontweight='bold', fontsize=15)

axes_list = [[0, 0], [0, 1], [1, 0], [1, 1]]
df_list = [df_w0_pca0, df_w0_pca1, df_w1_pca0, df_w1_pca1]

for i in range(len(axes_list)):
    plot_df_line(ax[axes_list[i][0], axes_list[i][1]],      #Mentioning Axis of figure
                    df_list[i], 'total_window', 'recall',   #Dataframe, params to be plotted
                    regress=True,
                    title=f'(W: {axes_list[i][0]}, pca: {axes_list[i][1]}, L: 45)', 
                    #yrange=(0, 0.5)
                )

-----------------------